In [1]:
import mmi.mmi_client
import mmi.tracker as tr
import numpy as np

from bokeh.io import push_notebook, output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool
 
import threading

DEBUG:shapely.geos:Found GEOS DLL: <CDLL '/opt/conda/lib/python3.6/site-packages/shapely/.libs/libgeos_c-09576097.so.1.9.0', handle 15806c0 at 0x7faf2c3d7d30>, using it.
DEBUG:shapely.geos:Trying `CDLL(libc.so.6)`
DEBUG:shapely.geos:Library path: 'libc.so.6'
DEBUG:shapely.geos:DLL: <CDLL 'libc.so.6', handle 7faf527874e0 at 0x7faf50a9ac88>


In [2]:
time = -1

data = dict(
    H = None,
    zb = None,
    zs = None,
    xz = None,
)

animate = True

def get_data(model, t):
    
    global time, data
    
    time = t

    for var in data.keys():
        data[var] = model.get_var(var)[0]

                
def listen(model):
    
    global data
    
    t_old = model.get_current_time()
    while model.get_current_time() <= model.get_end_time():
        t_new = model.get_current_time()
        if (t_old != t_new):
            
            get_data(model, t_new)
            
            if animate:
                update_plot()
                
        t_old = t_new
    time.sleep(1)
    
    
def update_plot():
    
    global time, data
    
    try:
        p.title.text = ('t = %0.2f s' % time)
        waterline.data_source.data['y'] = data['zs'] + data['H']
    except:
        p.title.text = ('t = %0.2f s, does not exist yet' % time)
        
    push_notebook()

In [3]:
model = mmi.mmi_client.MMIClient('tcp://xbeach-orchestrator:50010')

In [4]:
get_data(model, 0)

In [5]:
output_notebook()

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select,lasso_select"
p = figure(tools=TOOLS, 
           width=700, 
           height=350, 
           title='t = ??? s', 
           x_axis_label='distance in x-direction [m]', 
           y_axis_label='height [m+NAP]')

bath = p.line(data['xz'], data['zb'], line_width=2, color='black')
waterline = p.line(data['xz'], data['zs'] + data['H'], color='blue')

hover = HoverTool(renderers=[waterline])

hover.tooltips = [
    ('water level', '$y')
]
p.add_tools(hover)

show(p, notebook_handle=True)

Loading BokehJS ...

In [6]:
thread = threading.Thread(target=lambda: listen(model))
thread.start()